In [1]:
import os
import sys
sys.path.append('../')

from library import Graph, GraphLib

In [3]:
graphLib = GraphLib(config='./config_test.yaml')
print(graphLib)

Graph Library with configurations:
{'dataset': 'CIFAR10', 'data_dir': '/scratch/gpfs/stuli/pytorch_datasets/', 'manual_dir': None, 'train_batch_size': 256, 'test_batch_size': 512, 'module_vertices': 3, 'head_vertices': 6, 'max_edges': 9, 'max_modules': 2, 'base_ops': ['conv5x5-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3'], 'flatten_ops': ['global-avg-pool', 'flatten'], 'dense_ops': ['dense-1024', 'dense-4096', 'dropout-p5', 'dropout-p4'], 'hash_algo': 'sha256'}
Number of graphs: 0


In [4]:
graphLib.build_library()

Generating modules...
Using 3 vertices, 4 labels, max 9 edges
	Generated up to 3 vertices: 8 modules (8 without hashing)

Generating heads...
Using 6 vertices, 6 labels
	Generated up to 4 vertices: 2 heads (2 without hashing)
	Generated up to 5 vertices: 10 heads (10 without hashing)
	Generated up to 6 vertices: 42 heads (42 without hashing)

Generating graphs...
Using max 2 modules



	Generated up to 1 modules: 336 graphs



	Generated up to 2 modules: 3024 graphs
Graph library created! 
3024 graphs within the design space.


In [5]:
graphLib.save_dataset('./dataset.json')

Dataset saved to: ./dataset.json


In [6]:
graphLib_new = GraphLib.load_from_dataset('./dataset.json')
graphLib_new

Graph Library with configurations:
{'dataset': 'CIFAR10', 'data_dir': '/scratch/gpfs/stuli/pytorch_datasets/', 'manual_dir': None, 'train_batch_size': 256, 'test_batch_size': 512, 'module_vertices': 3, 'head_vertices': 6, 'max_edges': 9, 'max_modules': 2, 'base_ops': ['conv5x5-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3'], 'flatten_ops': ['global-avg-pool', 'flatten'], 'dense_ops': ['dense-1024', 'dense-4096', 'dropout-p5', 'dropout-p4'], 'hash_algo': 'sha256'}
Number of graphs: 3024

In [7]:
graphLib_new.library[0]

Model parameters: None
Accuracies: {'train': None, 'val': None, 'test': None}
Embedding: None
Hash: dc1bcce91c615b8b737be91bbf298911e444764a4326cf339dc9e8f88d3ade1a
Module:
[[0 1 0]
 [0 0 1]
 [0 0 0]]
Labels:['input', 'conv5x5-bn-relu', 'output']
Module:
[[0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]
Labels:['input', 'global-avg-pool', 'dense_classes', 'output']